# 📦 Task 3: Convert Dataset to YOLO Format

## 🎯 Objective
Convert the Waste Classification dataset (image classification format) to YOLO object detection format.

---

## 📚 Theory: YOLO Data Format

### Classification vs Detection Format

| Aspect | Classification | Detection (YOLO) |
|--------|---------------|------------------|
| Task | What is in image? | What + Where? |
| Output | Class label | Class + Bounding box |
| Annotation | Folder structure | .txt files |

### YOLO Annotation Format
Each image has a corresponding `.txt` file with:
```
<class_id> <x_center> <y_center> <width> <height>
```

**All values are normalized [0, 1]:**
```
x_center = (x_min + x_max) / (2 × image_width)
y_center = (y_min + y_max) / (2 × image_height)
width = (x_max - x_min) / image_width
height = (y_max - y_min) / image_height
```

### Our Approach
Since this is a **classification dataset** (no bounding boxes), we'll:
1. Use the **entire image as the bounding box** (object fills frame)
2. Or use a slight padding margin

This is valid because waste items typically fill most of the image in this dataset.

In [ ]:
import numpy as np
import os
import shutil
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm.notebook import tqdm
import yaml
import random

print("✅ Libraries imported!")

In [ ]:
# Project paths
PROJECT_ROOT = Path(r"D:\het\SELF\RP\YOLO-V11-PRO")
RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

# Find dataset
DATASET_DIR = None
for p in [RAW_DATA_DIR / "DATASET", RAW_DATA_DIR]:
    if (p / "TRAIN").exists():
        DATASET_DIR = p
        break

if DATASET_DIR:
    TRAIN_DIR = DATASET_DIR / "TRAIN"
    TEST_DIR = DATASET_DIR / "TEST"
    print(f"✅ Dataset found: {DATASET_DIR}")
else:
    print("❌ Dataset not found!")

# Output directories
YOLO_DIR = PROCESSED_DIR
YOLO_IMAGES_TRAIN = YOLO_DIR / "images" / "train"
YOLO_IMAGES_VAL = YOLO_DIR / "images" / "val"
YOLO_LABELS_TRAIN = YOLO_DIR / "labels" / "train"
YOLO_LABELS_VAL = YOLO_DIR / "labels" / "val"

# Create directories
for d in [YOLO_IMAGES_TRAIN, YOLO_IMAGES_VAL, YOLO_LABELS_TRAIN, YOLO_LABELS_VAL]:
    d.mkdir(parents=True, exist_ok=True)

print(f"\n📁 YOLO Output Structure:")
print(f"   {YOLO_DIR}")
print(f"   ├── images/train/")
print(f"   ├── images/val/")
print(f"   ├── labels/train/")
print(f"   └── labels/val/")

## 📐 Bounding Box Mathematics

### Coordinate Systems

**Pixel Coordinates (Absolute):**
```
(0,0) ─────────────────► x (width)
  │
  │    ┌─────────┐
  │    │ (x₁,y₁) │
  │    │         │
  │    │ (x₂,y₂) │
  │    └─────────┘
  ▼
  y (height)
```

**YOLO Format (Normalized Center):**
```
x_center = (x₁ + x₂) / (2 × W)
y_center = (y₁ + y₂) / (2 × H)
box_w = (x₂ - x₁) / W
box_h = (y₂ - y₁) / H
```

In [ ]:
# ============================================================
# BOUNDING BOX CONVERSION FUNCTIONS (NumPy)
# ============================================================

def xyxy_to_yolo(x1, y1, x2, y2, img_width, img_height):
    """
    Convert absolute box coordinates to YOLO format.
    
    Input: (x1, y1, x2, y2) - top-left and bottom-right corners
    Output: (x_center, y_center, width, height) - normalized [0,1]
    """
    # Calculate center
    x_center = (x1 + x2) / 2.0 / img_width
    y_center = (y1 + y2) / 2.0 / img_height
    
    # Calculate normalized dimensions
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    
    return x_center, y_center, width, height

def yolo_to_xyxy(x_center, y_center, width, height, img_width, img_height):
    """
    Convert YOLO format back to absolute coordinates.
    """
    # Denormalize
    x_center_abs = x_center * img_width
    y_center_abs = y_center * img_height
    w_abs = width * img_width
    h_abs = height * img_height
    
    # Calculate corners
    x1 = x_center_abs - w_abs / 2
    y1 = y_center_abs - h_abs / 2
    x2 = x_center_abs + w_abs / 2
    y2 = y_center_abs + h_abs / 2
    
    return int(x1), int(y1), int(x2), int(y2)

def create_full_image_bbox(img_width, img_height, margin=0.02):
    """
    Create bounding box covering (almost) entire image.
    
    margin: Percentage padding from edges (0.02 = 2%)
    """
    x1 = int(margin * img_width)
    y1 = int(margin * img_height)
    x2 = int((1 - margin) * img_width)
    y2 = int((1 - margin) * img_height)
    
    return xyxy_to_yolo(x1, y1, x2, y2, img_width, img_height)

# Test
test_bbox = create_full_image_bbox(640, 480, margin=0.05)
print(f"✅ Test bbox (normalized): {test_bbox}")
print(f"   Sum should be ~1.8: {sum(test_bbox):.3f}")

In [ ]:
# Visualize bounding box concept
def visualize_bbox_conversion():
    """Show how bounding box conversion works."""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle('📐 Bounding Box Coordinate Systems', fontsize=14, fontweight='bold')
    
    # Sample image dimensions
    W, H = 640, 480
    
    # Absolute coordinates (xyxy)
    x1, y1, x2, y2 = 100, 80, 540, 400
    
    # Convert to YOLO
    xc, yc, w, h = xyxy_to_yolo(x1, y1, x2, y2, W, H)
    
    # Plot 1: Absolute coordinates
    ax1 = axes[0]
    ax1.set_xlim(0, W)
    ax1.set_ylim(H, 0)  # Invert y-axis
    rect1 = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                               linewidth=3, edgecolor='red', facecolor='none')
    ax1.add_patch(rect1)
    ax1.plot(x1, y1, 'go', markersize=10, label=f'Top-left ({x1},{y1})')
    ax1.plot(x2, y2, 'bo', markersize=10, label=f'Bottom-right ({x2},{y2})')
    ax1.set_title('Absolute Coordinates (xyxy)')
    ax1.set_xlabel(f'x (width = {W} px)')
    ax1.set_ylabel(f'y (height = {H} px)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: YOLO normalized
    ax2 = axes[1]
    ax2.set_xlim(0, 1)
    ax2.set_ylim(1, 0)
    rect2 = patches.Rectangle((xc - w/2, yc - h/2), w, h,
                               linewidth=3, edgecolor='green', facecolor='none')
    ax2.add_patch(rect2)
    ax2.plot(xc, yc, 'r*', markersize=15, label=f'Center ({xc:.3f},{yc:.3f})')
    ax2.set_title(f'YOLO Format: {xc:.3f} {yc:.3f} {w:.3f} {h:.3f}')
    ax2.set_xlabel('Normalized x [0,1]')
    ax2.set_ylabel('Normalized y [0,1]')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'bbox_conversion.png', dpi=150)
    plt.show()

visualize_bbox_conversion()

In [ ]:
# ============================================================
# DATASET CONVERSION
# ============================================================

# Class mapping
CLASS_MAPPING = {
    'O': 0,  # Organic
    'R': 1   # Recyclable
}

CLASS_NAMES = ['Organic', 'Recyclable']

def convert_to_yolo_format(source_dir, dest_images_dir, dest_labels_dir, 
                           class_mapping, margin=0.02):
    """
    Convert classification dataset to YOLO detection format.
    
    For each image:
    1. Copy image to dest_images_dir
    2. Create label file with full-image bounding box
    """
    source_dir = Path(source_dir)
    dest_images_dir = Path(dest_images_dir)
    dest_labels_dir = Path(dest_labels_dir)
    
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    converted = 0
    errors = 0
    
    for class_folder in source_dir.iterdir():
        if not class_folder.is_dir():
            continue
            
        class_name = class_folder.name.upper()
        if class_name not in class_mapping:
            print(f"⚠️ Unknown class: {class_name}")
            continue
        
        class_id = class_mapping[class_name]
        
        images = [f for f in class_folder.iterdir() 
                 if f.suffix.lower() in image_extensions]
        
        for img_path in tqdm(images, desc=f"Converting {class_name}"):
            try:
                # Get image dimensions
                with Image.open(img_path) as img:
                    width, height = img.size
                
                # Generate YOLO bbox (full image with margin)
                xc, yc, bw, bh = create_full_image_bbox(width, height, margin)
                
                # Create unique filename
                new_name = f"{class_name}_{img_path.stem}"
                
                # Copy image
                dest_img_path = dest_images_dir / f"{new_name}{img_path.suffix}"
                shutil.copy2(img_path, dest_img_path)
                
                # Create label file
                label_path = dest_labels_dir / f"{new_name}.txt"
                with open(label_path, 'w') as f:
                    f.write(f"{class_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")
                
                converted += 1
                
            except Exception as e:
                print(f"Error: {img_path} - {e}")
                errors += 1
    
    return converted, errors

print("✅ Conversion function defined")

In [ ]:
# Convert training data
if DATASET_DIR:
    print("\n🔄 Converting TRAINING data...")
    train_converted, train_errors = convert_to_yolo_format(
        TRAIN_DIR, YOLO_IMAGES_TRAIN, YOLO_LABELS_TRAIN, CLASS_MAPPING
    )
    print(f"✅ Training: {train_converted} converted, {train_errors} errors")
    
    print("\n🔄 Converting TEST data (as validation)...")
    val_converted, val_errors = convert_to_yolo_format(
        TEST_DIR, YOLO_IMAGES_VAL, YOLO_LABELS_VAL, CLASS_MAPPING
    )
    print(f"✅ Validation: {val_converted} converted, {val_errors} errors")

In [ ]:
# ============================================================
# CREATE dataset.yaml
# ============================================================

dataset_config = {
    'path': str(YOLO_DIR.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'names': {
        0: 'Organic',
        1: 'Recyclable'
    }
}

yaml_path = YOLO_DIR / 'dataset.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print(f"✅ Created: {yaml_path}")
print("\n📄 dataset.yaml contents:")
print("="*40)
with open(yaml_path, 'r') as f:
    print(f.read())

In [ ]:
# ============================================================
# VERIFY CONVERSION
# ============================================================

def verify_yolo_dataset(images_dir, labels_dir, n_samples=3):
    """Visualize sample images with their bounding boxes."""
    
    images_dir = Path(images_dir)
    labels_dir = Path(labels_dir)
    
    images = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png"))
    samples = random.sample(images, min(n_samples, len(images)))
    
    fig, axes = plt.subplots(1, n_samples, figsize=(5*n_samples, 5))
    if n_samples == 1:
        axes = [axes]
    
    colors = {0: 'green', 1: 'blue'}
    
    for ax, img_path in zip(axes, samples):
        # Load image
        img = np.array(Image.open(img_path))
        ax.imshow(img)
        
        # Load label
        label_path = labels_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    xc, yc, w, h = map(float, parts[1:])
                    
                    # Convert to absolute coordinates
                    img_h, img_w = img.shape[:2]
                    x1, y1, x2, y2 = yolo_to_xyxy(xc, yc, w, h, img_w, img_h)
                    
                    # Draw rectangle
                    rect = patches.Rectangle(
                        (x1, y1), x2-x1, y2-y1,
                        linewidth=2, edgecolor=colors[class_id], facecolor='none'
                    )
                    ax.add_patch(rect)
                    ax.text(x1, y1-5, CLASS_NAMES[class_id], 
                           color=colors[class_id], fontsize=10, fontweight='bold',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax.set_title(img_path.name[:30])
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'yolo_format_samples.png', dpi=150)
    plt.show()

print("\n🔍 Verifying Training Samples:")
verify_yolo_dataset(YOLO_IMAGES_TRAIN, YOLO_LABELS_TRAIN, n_samples=4)

In [ ]:
# Dataset statistics
def count_yolo_dataset(images_dir, labels_dir):
    """Count images and verify labels."""
    images = list(Path(images_dir).glob("*.*"))
    labels = list(Path(labels_dir).glob("*.txt"))
    
    # Count per class
    class_counts = {0: 0, 1: 0}
    for label_path in labels:
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.strip().split()[0])
                class_counts[class_id] += 1
    
    return len(images), len(labels), class_counts

train_stats = count_yolo_dataset(YOLO_IMAGES_TRAIN, YOLO_LABELS_TRAIN)
val_stats = count_yolo_dataset(YOLO_IMAGES_VAL, YOLO_LABELS_VAL)

print("\n📊 YOLO Dataset Statistics:")
print("="*50)
print(f"\n🏋️ Training Set:")
print(f"   Images: {train_stats[0]}")
print(f"   Labels: {train_stats[1]}")
print(f"   Organic: {train_stats[2][0]}, Recyclable: {train_stats[2][1]}")

print(f"\n🧪 Validation Set:")
print(f"   Images: {val_stats[0]}")
print(f"   Labels: {val_stats[1]}")
print(f"   Organic: {val_stats[2][0]}, Recyclable: {val_stats[2][1]}")

## 📝 Summary

### Conversion Complete!

| Component | Location |
|-----------|----------|
| Training Images | `data/processed/images/train/` |
| Training Labels | `data/processed/labels/train/` |
| Validation Images | `data/processed/images/val/` |
| Validation Labels | `data/processed/labels/val/` |
| Config | `data/processed/dataset.yaml` |

### YOLO Label Format:
```
<class_id> <x_center> <y_center> <width> <height>
```

### Next: Task 4 - Deeper EDA with the new format

In [ ]:
print("\n" + "="*60)
print("✅ TASK 3 COMPLETE: Dataset Converted to YOLO Format")
print("="*60)
print("\n📋 What was accomplished:")
print("   ✓ Bounding box math explained")
print("   ✓ xyxy_to_yolo() conversion function")
print("   ✓ yolo_to_xyxy() inverse function")
print("   ✓ Training images converted")
print("   ✓ Validation images converted")
print("   ✓ dataset.yaml created")
print("   ✓ Conversion verified visually")
print("\n➡️ Ready for Task 4: Deeper Data Visualization")